In [18]:
import os
import pickle
import pprint
from glob import glob
os.chdir('/home/users/alexno/sleep-staging')

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from h5py import File

from utils.plotting import plot_psg_hypnogram_hypnodensity
from utils.plotting import plot_hypnodensity

sns.set(style='white', context='poster', rc={'figure.figsize':(14,10)})

In [3]:
available_predictions = glob("experiments/**/SSC-WSC_test_predictions.pkl", recursive=True)
pprint.pprint(available_predictions)

['experiments/massc/avg_kw21_rnn128/20201209_025443/SSC-WSC_test_predictions.pkl',
 'experiments/massc/att30/20201124_080609/SSC-WSC_test_predictions.pkl',
 'experiments/massc/avg_kw21_rnn128_cyclic/20201212_150902/SSC-WSC_test_predictions.pkl',
 'experiments/massc/avg_kw21/20201126_043826/SSC-WSC_test_predictions.pkl']


In [4]:
selected_experiment = available_predictions[-1]
with open(selected_experiment, 'rb') as f:
    predictions = pickle.load(f)
list_studies = list(predictions.keys())

In [5]:
selected_record = 'SSC_6542_1.h5'
record = predictions[selected_record]
target = record['true'].argmax(-1).repeat(30)
logits = record['logits']
seqs = record['seq_nr']
ss = record['stable_sleep']
print(record.keys())
print('logits.shape:', logits.shape)
print('target.shape:', target.shape)
print('seqs.shape:', seqs.shape)
print('ss.shape:', ss.shape)

dict_keys(['true', 'true_label', 'predicted', 'predicted_label', 'stable_sleep', 'logits', 'seq_nr'])
logits.shape: (29100, 5)
target.shape: (29100,)
seqs.shape: (97,)
ss.shape: (970,)


In [19]:
target = []
logits = []
for idx, (r, d) in enumerate(predictions.items()):
    if idx < 50:
        target.append(d['true'].argmax(-1).repeat(30))
        logits.append(d['logits'])
    else:
        break
target = np.concatenate(target, axis=0)
logits = np.concatenate(logits, axis=0)
print('logits.shape:', logits.shape)
print('target.shape:', target.shape)

logits.shape: (1375800, 5)
target.shape: (1375800,)


In [ ]:
import umap
import umap.plot


mapper = umap.UMAP().fit(logits)

In [ ]:
umap.plot.points(mapper, labels=target)

In [ ]:
from sklearn.manifold import TSNE
logits_embedded = TSNE(n_components=2, n_jobs=64).fit_transform(logits)
print(logits_embedded.shape)

plt.figure(dpi=300)
hyp_dict = {0: 'W', 1: 'N1', 2: 'N2', 3: 'N3', 4: 'R'}
for c in np.unique(target):
    plt.scatter(logits_embedded[target == c, 0], logits_embedded[target == c, 1], s=0.5, label=hyp_dict[c])
plt.legend()

In [ ]:
plt.figure(dpi=300)
for c in np.unique(target):
    plt.scatter(logits_embedded[target == c, 0], logits_embedded[target == c, 1], s=0.5, label=c)
plt.legend()